In [4]:
import numpy as np
from scipy import stats
from numpy.linalg import inv

In [2]:
def rvs(mu, Sigma):
    z = stats.norm.rvs(size = len(mu))
    return mu + np.linalg.cholesky(Sigma).dot(z)

def rvs(mu, Sigma):
    # from stober https://gist.github.com/stober/4964727
    # better numerical stability than choleskey for near-singular Sigma
    z = stats.norm.rvs(size = len(mu))
    [U, S, V] = np.linalg.svd(Sigma)
    A = U * np.sqrt(S)
    return mu + A.dot(z)

mvn = stats.multivariate_normal

In [ ]:
def marginal(mu, Sigma, i):
    mu = np.array(mu)[i]
    Sigma = np.array(Sigma)[i, i]
    return mvn(mu, Sigma)

In [75]:
def conditional(mu, Sigma, x):
    mu = np.array(mu)
    Sigma = np.array(Sigma)
    unobserved = np.isnan(x)
    observed = np.logical_not(unobserved)
    mu1 = mu[unobserved]
    mu2 = mu[observed]
    x2 = np.array(x)[observed]
    invSigma22 = inv(Sigma[np.ix_(observed, observed)])
    Sigma11 = Sigma[np.ix_(unobserved, unobserved)]
    Sigma12 = Sigma[np.ix_(unobserved, observed)]
    Sigma21 = Sigma12.T
    mu = mu1 + Sigma12.dot(invSigma22).dot(x2 - mu2)
    Sigma = Sigma11 - Sigma12.dot(invSigma22).dot(Sigma21)
    return mvn(mu, Sigma)

In [76]:
# informational form, gives same results as above!

def conditional(mu, Sigma, x):
    mu = np.array(mu)
    Sigma = np.array(Sigma)
    Lambda = inv(Sigma)
    Epsilon = Lambda.dot(mu)
    
    unobserved = np.isnan(x)
    observed = np.logical_not(unobserved)
    mu1 = mu[unobserved]
    mu2 = mu[observed]
    x2 = np.array(x)[observed]
    
    Epsilon1 = Epsilon[unobserved]
    Lambda12 = Lambda[np.ix_(unobserved, observed)]
    Epsilon = Epsilon1 - Lambda12.dot(x2) 
    Lambda = Lambda[np.ix_(unobserved, unobserved)]
        
    Sigma = inv(Lambda)
    mu = Sigma.dot(Epsilon)
    return mvn(mu, Sigma)

In [81]:
mu = [3, 4, 5, 2]
Sigma = [
    [1,   0.5, 0.5, 0.5], 
    [0.5,  18, 0.5, 0.5], 
    [0.5, 0.5,  23, 0.5], 
    [0.5, 0.5, 0.5, 5]
]
x = [2, np.nan, np.nan, 2]

conditional(mu, Sigma, x).rvs()

array([ 5.32316305, -2.20281182])